In [ ]:
! pip install openai
!pip install -q diffusers transformers accelerate invisible-watermark>=0.2.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.3 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! cd /content/drive/MyDrive/AI

## File Processing

In [ ]:
import os
from pydub import AudioSegment

def split_mp3_to_segments(mp3_file, segment_length=15000):
    # Extract file name without extension
    file_name = os.path.splitext(mp3_file)[0]

    # Create a directory with the same name as the mp3 file
    if not os.path.exists(file_name):
        os.makedirs(file_name)

    # Load the mp3 file
    audio = AudioSegment.from_mp3(mp3_file)

    # Calculate the number of segments
    total_length = len(audio)
    num_segments = total_length // segment_length + (1 if total_length % segment_length != 0 else 0)

    # Split the audio and save the segments
    for i in range(num_segments):
        start_time = i * segment_length
        end_time = min((i + 1) * segment_length, total_length)
        segment = audio[start_time:end_time]
        segment_name = os.path.join(file_name, f"{file_name}_segment_{i+1}.mp3")
        segment.export(segment_name, format="mp3")
        print(f"Created {segment_name}")

# Example usage
mp3_file = "/content/drive/MyDrive/AI/music.mp3"
split_mp3_to_segments(mp3_file)


## MusicLLM Caption

In [ ]:
def get_captions():
    return [
        "This music sounds like a mysterious and atmospheric soundscape. It features a haunting melody played on a string instrument, accompanied by subtle, ambient sounds. The tempo is slow and deliberate, creating a sense of suspense and intrigue. The overall mood is dark and evocative, with a hint of melancholy.",
        "This music is a beautiful and ethereal soundscape. It features a haunting melody played on a string instrument, accompanied by gentle, ambient sounds. The tempo is slow and meditative, creating a sense of peace and tranquility. The overall mood is serene and dreamy, with a hint of sadness.",
        "This music is a blend of mystery and ethereal beauty. It starts with a haunting melody played on a string instrument, creating a sense of suspense and intrigue. As the music progresses, it becomes more atmospheric, with the addition of ambient sounds and a delicate piano melody. The tempo is slow and deliberate, creating a sense of peace and tranquility. The overall mood is dark and evocative, with a hint of melancholy",
        "This music is a captivating blend of haunting melodies and ambient sounds. It begins with a slow, deliberate tempo, creating a sense of mystery and intrigue. A string instrument plays a haunting melody, while subtle ambient sounds add depth and texture to the soundscape. The music gradually builds in intensity, with the string instrument becoming more prominent and the ambient sounds becoming more layered. The overall mood is dark and evocative, with a hint of melancholy."
    ]

##Recurrent LLM

In [ ]:
from openai import OpenAI
import os
import re
from diffusers import AutoPipelineForText2Image
import torch

pipeline_text2image = AutoPipelineForText2Image.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0", torch_dtype=torch.float16, variant="fp16", use_safetensors=True
).to("cuda")

# OpenAI API key setup
client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY", 'sC8itCICn0hoUZvBRKvQfDvq-HlrPIKJhOCZThlsA'))

# Embedded prompt template
PROMPT_TEMPLATE = """
You are an expert in story writing who can fully understand music and realize synesthesia. Please design a storyboard for a piece of music to express the music. You only need to design the story and the description of the frame instead of designing the real frame.
Please generate artistic story based on the music. Make sure you accurately describe the current frame, the current frame is an image, you need to express the story in the image, the description should not be too long, but should be prepared to express emotion and related to the image of the previous frame.

In the design of each frame described in this storyboard, you will receive the following input：
input 1. A description of 10-second clip of the current music
input 2. The summary of the entire music
input 3. The summary of the previous story
input 4. The description of the previous frame
input 5. Optional suggestions when designed the next frame

During the design process of current frame, make sure you always keep the following guidelines in mind:
    1. Coherence between frames: The current frame is coherent with the previous frame
    2. Alignment of music clip and frame: The current frame can realie synesthesia with the input music
    3. Consistency of music and story: story as a whole fits the overall and theme of the music
    4. Integrity of a single frame description: The whole story is coherent, but while maintaining continuity between frames, each frame itself is also complete
    5. For the description of the current frame, please do not try to show continuous changes within a description, a frame cannot show changes like a video

The following is the input you receive, if only the description of the current music clip is left blank, it means that it is currently the first frame of the entire storyboard. For the first frame, use the summary of the current music clip and the summary of the current frame directly as a summary of the entire music and story to start the iteration.
Input 1 - Description for current music clip: {current_clip}
Input 2 - Summary of the overall music: {music_summary}
Input 3 - Summary of the previous story: {previous_story}
Input 4 - Description the previous frame: {previous_frame}
Input 5 - Optional suggestions for the previous frame: {previous_suggestions}

Follow the template below to output the result, and replace the placeholder with your response:

***Template Begins***

## Analysis for current clip ##
[Think step by step. Based on the input of a 10-second clip of the current music, use a short paragraph to describe how you should design a picture to achieve synesthesia with music]

## Summary of Overall Music ##
[Based on the current music clip and the overall summary of the current music, update the summary of the current entire music and the theme]

## Description of Current Frame ##
[Based on the analysis of the current music clip, as well as the description of the previous frame and the summary of the previous story and optional suggestions, generate a description of the new frame of the storyboard]

## Suggestions for the Next Frame ##
[Based on the current frame you describe, make a suggestion for the design of the next frame in order to ensure that the next frame is coherent with the current frame and that the story as a whole fits the overall and theme of the music]

## Updated Story Theme Summary ##
[Based on the description of current frame and the summary of the previous story, update the summary of the overall direction and theme of the story]

***Template Ends***“”“"""

def img_generation(prompt, save_dir=' /content/drive/MyDrive/AI'):
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    image =  pipeline_text2image(prompt=prompt).images[0]
    save_path = os.path.join(save_dir, f"generated_image_{len(os.listdir(save_dir)) + 1}.png")

    image.save(save_path)
    print(f"Image saved to: {save_path}")
    return save_path

def recurrentStoryboard(current_clip, music_summary, previous_summary, previous_frame, previous_suggestions):
    try:
        completion = client.chat.completions.create(
            model="gpt-4",
            temperature=0.7,
            messages=[
                {"role": "user", "content": PROMPT_TEMPLATE.format(
                    current_clip=current_clip,
                    music_summary=music_summary,
                    previous_story=previous_summary,
                    previous_frame=previous_frame,
                    previous_suggestions=previous_suggestions
                )}
            ]
        )
        return completion.choices[0].message.content
    except Exception as e:
        return f"Error: {str(e)}"

def extract_info(response):
    music_summary = re.search(r"## Summary of Overall Music ##\n(.*?)\n\n", response, re.DOTALL).group(1).strip()
    previous_summary = re.search(r"## Updated Story Theme Summary ##\n(.*?)\n\n", response, re.DOTALL).group(1).strip()
    previous_frame = re.search(r"## Description of Current Frame ##\n(.*?)\n\n", response, re.DOTALL).group(1).strip()
    previous_suggestions = re.search(r"## Suggestions for the Next Frame ##\n(.*?)$", response, re.DOTALL).group(1).strip()
    return music_summary, previous_summary, previous_frame, previous_suggestions

def main():
    captions = get_captions()
    music_summary = ""
    previous_summary = ""
    previous_frame = ""
    previous_suggestions = ""

    for i, current_clip in enumerate(captions):
        print(f"\nProcessing caption {i+1}/{len(captions)}")
        response = recurrentStoryboard(current_clip, music_summary, previous_summary, previous_frame, previous_suggestions)
        print(response)
        music_summary, previous_summary, previous_frame, previous_suggestions = extract_info(response)
        img_generation(previous_frame)

if __name__ == "__main__":
    main()

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

model_index.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

Fetching 19 files:   0%|          | 0/19 [00:00<?, ?it/s]

model.fp16.safetensors:   0%|          | 0.00/1.39G [00:00<?, ?B/s]

...

Image saved to:  /content/drive/MyDrive/AI/generated_image_3.png

Processing caption 4/4


The following part of your input was truncated because CLIP can only handle sequences up to 77 tokens: ['and wisdom, and his eyes hold a profound melancholy. the forest in the background remains a dark and impenetrable wall, adding a touch of suspense and uncertainty.']
The following part of your input was truncated because CLIP can only handle sequences up to 77 tokens: ['and wisdom, and his eyes hold a profound melancholy. the forest in the background remains a dark and impenetrable wall, adding a touch of suspense and uncertainty.']


***Template Begins***

## Analysis for current clip ##
The current music clip maintains the suspenseful and atmospheric tone, yet introduces a touch of tranquility and serenity. The haunting melody of the string instrument plays a significant role, with the ambient sounds adding depth and texture to the soundscape. To achieve synesthesia with this music, the frame must balance the elements of suspense and tranquility, possibly introducing an element of revelation or discovery.

## Summary of Overall Music ##
The overall music continues to be a mysterious, suspenseful, and atmospheric soundscape with a strong sense of melancholy and dark undertones. The current clip introduces a layer of serenity and tranquility to the otherwise suspenseful narrative, suggesting a delicate balance between suspense and tranquility. There's an inclination towards revelation, reflecting the prominence of the string instrument in the current clip.

## Description of Current Frame ##
In this frame, the figur

  0%|          | 0/50 [00:00<?, ?it/s]

Image saved to:  /content/drive/MyDrive/AI/generated_image_4.png
